In [ ]:
import numpy as np
import os
import sys
import math
from PIL import Image
from matplotlib.pyplot import figure
import pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import torch
from torch import nn
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader


class StageOne(nn.Module):
    '''
    Stage One of Two Stage DNN.
  '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(256, 256),
            nn.Sigmoid(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            # nn.Linear(64,32),
            # nn.ReLU(),
            nn.Linear(64,16),
            nn.Sigmoid(),
        )


    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

class StageTwo(nn.Module):
    '''
    Stage Two of Two Stage DNN.
  '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
        nn.Linear(16, 16),
        nn.Sigmoid(),
        nn.Linear(16,512),
        nn.Tanh(),
        nn.Linear(512,256),
        nn.Tanh(),
        nn.Linear(256,256),
        #nn.ReLU(),
        #nn.Linear(256,256),
        nn.Sigmoid(),
        )


    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)  



def add_noise(outputVector,noise_percent,stdev):
    '''
    Add noise to the output vector.
    '''
    mean = 0
    noise = np.random.normal(mean,stdev,outputVector.shape) * noise_percent
    return outputVector + noise

def Noisy_Testing(stdev, testRounds, noiseCrossSection, stageOneModel, stageTwoModel, inputImageVectors, outputVector):
    '''
    Test the DNN with noise.
    '''
    noisySetDisplayArray=[[0]*2*len(stdev) for i in range(testRounds)]
    plotObject = {'fh': [], 'ffa': []}
    for j in range(len(stdev)):
        for k in range(testRounds) :
            corruptedVector = add_noise(inputImageVectors[k],noiseCrossSection,stdev[j])
            noisySetDisplayArray[k][2*j] = getPredictedImage(corruptedVector) 
            stageOnePrediction = stageOneModel(torch.from_numpy(corruptedVector.astype('float32'))).detach().numpy()
            stageTwoPrediction = stageTwoModel(torch.from_numpy(stageOnePrediction.astype('float32'))).detach().numpy()
            for l in range(256):
                if stageTwoPrediction[l] > 0.1:
                    stageTwoPrediction[l] = 1
                else:
                    stageTwoPrediction[l] = 0
            noisySetDisplayArray[k][(2*j)+1] = getPredictedImage(stageTwoPrediction)
            fh,ffa = calculate_performance_metrics(outputVector[k],stageTwoPrediction,'default')
            plotObject['fh'].append(round(fh,2))
            plotObject['ffa'].append(round(ffa,2))

    x = np.array([
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
        0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
        0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
        0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02,
        0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03,
        0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,0.1])

    fig, ax = plt.subplots(figsize=(10,8))
        
    ax.scatter(x, plotObject['fh'], label="Fh", marker= "^", color="orange", s=30)
    ax.scatter(x, plotObject['ffa'], label="Ffa", marker= "^", color="red", s=30)
    
    plt.ylim(0, 1)
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(tick.FormatStrFormatter('%g'))
    
    
    plt.xlabel('Gaussian Noise Level (stdev, at ' + str(noiseCrossSection*100) + ' pct xsecn)')
    plt.ylabel("$\it{Fh}$ and $\it{Ffa}$")
    plt.xticks(x)
    plt.title('Graph of Fh and Ffa vs. Noise (Cross-section '+ str(int(noiseCrossSection*100)) + '%) Standard Deviation \n for noise-corrupted Alphanumeric Imagery (16x16 pixels) for \n Heteroassociative Deep Neural Network')
    plt.legend()
    plt.show()
    displayNoisyPredictions(noisySetDisplayArray,stdev,noiseCrossSection)
            

def Create_Image_Set(filename, ASL):
    ImageVectors = []
    if filename!='ASL32x' and filename!='ASL64x' and ASL==False: #set1 and set2
        for i in range(10):
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, str(i) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))
        for i in range(26):
            x = i + 65
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))

    return np.array(ImageVectors)

def calculate_performance_metrics(inputVector, outputVector, context):
    if context == 'semantics':
        totalBlackPixelCount = sum(x == 1 for x in inputVector)
        totalWhitePixelCount = sum(x == 0 for x in inputVector)
    elif context == 'default':
        totalBlackPixelCount = sum(x == 0 for x in inputVector)
        totalWhitePixelCount = sum(x == 1 for x in inputVector)

    wrongBlackPixelCount = 0
    rightBlackPixelCount = 0
    
    for i in range(len(inputVector)):
        if context == 'semantics':
            if outputVector[i] == 1:
                if  abs(outputVector[i] - inputVector[i]) < 0.0001:
                    rightBlackPixelCount += 1
                else:
                    wrongBlackPixelCount += 1
        elif context == 'default':
            if outputVector[i] < 0.001:
                if  abs(outputVector[i] - inputVector[i]) < 0.0001:
                    rightBlackPixelCount += 1
                else:
                    wrongBlackPixelCount += 1  

    fh = rightBlackPixelCount/totalBlackPixelCount
    ffa = wrongBlackPixelCount/totalWhitePixelCount
    return fh, ffa
    
def getPredictedImage(predictionVector):
        '''
        Get the predicted image.
        '''
        tempArray = []
        for i in range(len(predictionVector)):
            if predictionVector[i] < 0.0001:
                tempArray.append((0,0,0)) 
            else:
                tempArray.append((255,255,255))  
        row_length = int(math.sqrt(len(predictionVector)))
        finalImage = []
        for i in range(0,len(tempArray),row_length):
            finalImage.append(tempArray[i:i+(row_length-1)])
        imageArray = np.array(finalImage, dtype=np.uint8)
        return Image.fromarray(imageArray)

def displayNoisyPredictions(predictionDataArray, stddev, noiseCrossSection):
    '''
    Display the training predictions.
    '''
    f,axisArray = plt.subplots(len(predictionDataArray),len(predictionDataArray[0]),figsize=(30,30))
    plt.suptitle('Two Stage DNN Noisy Set performance with Noise Cross section '+ str(int(noiseCrossSection*100)) + '%', fontsize = 16)
    for i in range(len(predictionDataArray)):
        for j in range(len(predictionDataArray[0])):
            axisArray[i,j].axes.xaxis.set_visible(False)
            axisArray[i,j].axes.yaxis.set_visible(False)
            if j % 2 == 0:
                axisArray[i,j].set_title('Std. dev '+str(stddev[int(j/2)]), fontsize=9,x =0.5,y = 0.95)
            else:
                axisArray[i,j].set_title('Output', fontsize=9, x =0.5,y = 0.95)
            axisArray[i,j].imshow(predictionDataArray[i][j])

set1 = Create_Image_Set('characters1',False)
set2 = Create_Image_Set('characters2',False)
'''
    Sematic Representation of Dataset.
'''
'''
Sematic Representation Details:
Every character is represented as a 16-element array indicating the presence/absence of 16 different constituent elements
1st element - Large Upper left arc (0: absent, 1: present)
2nd element - Small Upper left arc (0: absent, 1: present)
3rd element - Large Upper right arc (0: absent, 1: present)
4th element - Small Upper right arc (0: absent, 1: present)
5th element - Large Lower left arc (0: absent, 1: present)
6th element - Small Lower left arc (0: absent, 1: present)
7th element - Large Lower right arc (0: absent, 1: present)
8th element - Small Lower right arc (0: absent, 1: present)
9th element - Large Backward slash (0: absent, 1: present)
10th element - Small Backward slash (0: absent, 1: present)
11th element - Large Forward slash (0: absent, 1: present)
12th element - Small Forward slash (0: absent, 1: present)
13th element - Large Vertical line (0: absent, 1: present)
14th element - Small Vertical line (0: absent, 1: present)
15th element - Large Horizontal line (0: absent, 1: present)
16th element - Small Horizontal line (0: absent, 1: present)
'''
outputArray = [ [0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0], #0
                [0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0], #1
                [0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0], #2
                [0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0], #3
                [0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0], #4
                [0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0], #5
                [1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0], #6
                [0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0], #7
                [0,1,0,1,0,1,0,1,1,0,1,0,0,0,0,0], #8
                [0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0], #9
                [0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1], #A
                [0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0], #B
                [1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0], #C
                [0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0], #D
                [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1], #E
                [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1], #F
                [1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1], #G
                [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0], #H
                [0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1], #I
                [0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0], #J
                [0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0], #K
                [0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0], #L
                [0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0], #M
                [0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0], #N
                [1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0], #O
                [0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0], #P
                [1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0], #Q
                [0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0], #R
                [0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0], #S
                [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1], #T
                [0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0], #U
                [0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0], #V
                [0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0], #W
                [0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0], #X
                [0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0], #Y
                [0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1] ] #Z

imageTensor = torch.Tensor(set1)
resultTensor = torch.Tensor(outputArray)
_dataSet = TensorDataset(imageTensor, resultTensor)
_dataLoader = DataLoader(_dataSet)

stageOneModel = StageOne()
criterion = nn.BCELoss()
optimizer = optim.Adam(stageOneModel.parameters(), lr=1e-3)

print("\n------EXTRA CREDIT - Stage 1------\n")

secondStageInput=[]

for epoch in range(100):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = stageOneModel(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

for i in range(36):
    output = stageOneModel(torch.from_numpy(set1[i].astype('float32'))).detach().numpy()
    for j in range(16):
        if output[j] > 0.01:
            output[j] = 1
        else:
            output[j] = 0
    # print("Output: ",output)
    fh, ffa = calculate_performance_metrics(outputArray[i], output,'semantics')
    # plt.figure()
    #plt.imshow(np.reshape(output,[16,16]))
    print("FH:", fh, "FFA:", ffa)
    secondStageInput.append(output)
    
# model.eval()

print("\n------EXTRA CREDIT - Stage 2------\n")

imageTensor = torch.Tensor(secondStageInput)
resultTensor = torch.Tensor(set1)
_dataSet = TensorDataset(imageTensor, resultTensor)
_dataLoader = DataLoader(_dataSet)

stageTwoModel = StageTwo()
criterion = nn.BCELoss()
optimizer = optim.Adam(stageTwoModel.parameters(), lr=1e-3)

for epoch in range(100):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = stageTwoModel(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

for i in range(36):
    output = stageTwoModel(torch.from_numpy(secondStageInput[i].astype('float32'))).detach().numpy()
    # print("Input:",set1[i],"Output:",output)
    for j in range(256):
        if output[j] > 0.1:
            output[j] = 1
        else:
            output[j] = 0
    # print("Output: ",output)
    fh, ffa = calculate_performance_metrics(set1[i], output,'default')
    # plt.figure()
    # plt.imshow(np.reshape(output,[16,16]))
    print("FH:", fh, "FFA:", ffa)

print("\n------EXTRA CREDIT - Noisy Testing------\n")
stdev = [0,0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05,0.1]
Noisy_Testing(stdev,36,0.1,stageOneModel,stageTwoModel,set1,set1)
Noisy_Testing(stdev,36,0.2,stageOneModel,stageTwoModel,set1,set1)
Noisy_Testing(stdev,36,0.25,stageOneModel,stageTwoModel,set1,set1)
Noisy_Testing(stdev,36,0.3,stageOneModel,stageTwoModel,set1,set1)
Noisy_Testing(stdev,36,0.35,stageOneModel,stageTwoModel,set1,set1)

